In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from sklearn.metrics import roc_auc_score
import riiideducation
from tqdm import tqdm
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
os.chdir('/kaggle/input/riiid-test-answer-prediction/')

In [ ]:
#import data
train = pd.read_csv('train.csv', nrows=10**6)
test = pd.read_csv("example_test.csv")
lectures = pd.read_csv("lectures.csv")
questions = pd.read_csv("questions.csv")

### Preprocessing

In [ ]:
#preprocessing
train[['prior_question_had_explanation']]=train[['prior_question_had_explanation']].replace(False,0).replace(True,1)

In [ ]:
trainQ = train[train['answered_correctly'] != -1]
trainLec = train[train['answered_correctly'] == -1]
def correctRatio(content_id:int) -> float: 
    global trainQ
    temp = trainQ[trainQ['content_id']==content_id]
    if(len(temp)== 0):
        return (-1,-1)
    return (temp['answered_correctly'].mean(), len(temp))

In [ ]:
cr = [] #correct_ratio
cn = [] #sample num
    
for i in tqdm(questions['question_id']):
    r,n = correctRatio(i)
    cr.append(r)
    cn.append(n)

In [ ]:
questions['sample_num'] = cn
questions['correct_ratio'] = cr


In [ ]:
df.apply(lambda x:x['tags'].split(),axis=1)

In [ ]:
questions[questions.apply(lambda x:type(x['tags']),axis=1)==float]

In [ ]:
questions.loc[questions.isna()['tags'],'tags'] = '-1'

questions['tags_list'] = questions.apply(lambda x:x['tags'].split(),axis=1)

In [ ]:
questionsValid = questions[(questions['correct_ratio'] !=-1) & (questions['sample_num']>=10)]

In [ ]:
questionsValid.sort_values("correct_ratio")

In [ ]:
def tagRatio(tagsDic,arr, n):
    temp = []
    for i in arr:
        i = tagsDic[i]
        if(i[1]>=n):
            temp.append(i[0])
            
    if(len(temp)==0):
        return -1
    else:
        return np.mean(temp)

In [ ]:
def getanswerRatioByTags(user_id):
    
    df = trainQ[trainQ['user_id']==user_id]

    temp = questions[['question_id','tags_list']]

    temp.columns = ['content_id','tags_list']

    df = df.merge(temp,how='left', on='content_id')

    tags = set(df['tags_list'].sum())

    tagsDic = {}
    for tag in tags:
        tagsDic[tag]=[]

    

  
    for i in range(len(df)):
        for j in df.iloc[i,10]:
            tagsDic[j].append(df.iloc[i,7])

    for k,v in tagsDic.items():
        tagsDic[k] = (np.mean(v),len(v))

    df['answer_ratio_by_tag'] = df.apply(lambda x:tagRatio(tagsDic,x['tags_list'],10),axis=1)
    return df[['user_id','content_id','timestamp','task_container_id','answer_ratio_by_tag']]


In [ ]:
X = pd.DataFrame()

In [ ]:
for user_id in tqdm(trainQ['user_id'].unique()):
    X = X.append(getanswerRatioByTags(user_id))
    

In [ ]:
usrCorrect = trainQ[['user_id','answered_correctly']].groupby("user_id").mean()
problemCorrect = trainQ[['content_id','answered_correctly']].groupby("content_id").mean()

In [ ]:
z = X.copy()

In [ ]:
X = z.copy()

In [ ]:
X = trainQ.copy()

In [ ]:
X[['timestamp']] = np.log(X[['timestamp']]+1)

In [ ]:
X[['task_container_id']] = np.log(X[['task_container_id']]+1)

In [ ]:
#using all elements
X = X.merge(usrCorrect.reset_index(), how='left',on='user_id')

X = X.merge(problemCorrect.reset_index(), how='left',on ='content_id')

In [ ]:
X = X[['answer_ratio_by_tag','answered_correctly_x','answered_correctly_y']]
X.columns = ['tag','user','problem']
y = trainQ['answered_correctly']

In [ ]:
X = X[['timestamp','task_container_id','answer_ratio_by_tag','answered_correctly_x','answered_correctly_y']]
X.columns = ['timestamp','task_container_id','tag','user','problem']
y = trainQ['answered_correctly']

In [ ]:
X = X[['answered_correctly_y','answered_correctly']]
X.columns = ['user','problem']
y = trainQ['answered_correctly']

In [ ]:
X.loc[X['tag']==-1,'tag'] = X.loc[X['tag']==-1,'user']

In [ ]:
from  sklearn.linear_model import LogisticRegression
from  sklearn.model_selection import train_test_split
Xt, Xv, Yt, Yv = train_test_split(X, y, test_size =0.2, shuffle=True)

Reg = LogisticRegression()
Reg.fit(Xt, np.array(Yt))

y_pred = Reg.predict_proba(Xv)[:,1]
#y_pred = np.array(Xv['answered_correctly_content'])
y_true = np.array(Yv)

roc_auc_score(y_true, y_pred)

In [ ]:
## lgbm 모델, 결과가 좋지않아 사용안함
# import lightgbm as lgb

# train_ds = lgb.Dataset(Xt, label = Yt) 
# test_ds = lgb.Dataset(Xv, label = Yv) 

# params = {'learning_rate': 0.01, 
#           'max_depth': 16, 
#           'boosting': 'gbdt', 
#           'objective': 'binary', 
#           'metric': 'auc', 
#           'is_training_metric': True, 
#           'num_leaves': 144, 
#           'feature_fraction': 0.9, 
#           'bagging_fraction': 0.7, 
#           'bagging_freq': 5, 
#           'seed':2021}

# model = lgb.train(params, train_ds, 1000, test_ds, verbose_eval=100, early_stopping_rounds=100)

### EDA

In [ ]:

sns.distplot(train.groupby("user_id").count()).set_title("The number of contents per id")

valid varialbe I think<br>
- user_id and content_id(base)
- timestamp or task_container_id
- prior_question_had_explanation



In [ ]:
train['timestamp_log'] = np.log(train['timestamp']+1)
train['task_container_id_log'] = np.log(train['task_container_id']+1)

train[train['answered_correctly']!=-1].corr()['answered_correctly']

### preprocessing

In [ ]:
usrCorrect = train[['user_id','answered_correctly']].groupby("user_id").mean()
problemCorrect = train[['content_id','answered_correctly']].groupby("content_id").mean()

np.random.seed(2020)
X = train.sample(10**7)

#using all elements
X = X.merge(usrCorrect.reset_index(), on='user_id')

X = X.merge(problemCorrect.reset_index(), on ='content_id')

X = X[['answered_correctly_y','answered_correctly','answered_correctly_x']]
X = X[X['answered_correctly_x'] != -1]
X.columns = ['usr','problem','y']
y = X['y']
X = X.iloc[:,:-1]

In [ ]:
# #using last element(tail)
# X = train.groupby(["user_id")

# X = X[X['answered_correctly'] != -1]

# X = X.merge(usrCorrect.reset_index(), on='user_id')

# X = X.merge(problemCorrect.reset_index(), on='content_id')

# y = X['answered_correctly_x']

# X = X[['answered_correctly_y','answered_correctly']]


# X.columns  = ['usr','problem']

### Modeling

In [ ]:
from  sklearn.linear_model import LogisticRegression
from  sklearn.model_selection import train_test_split
Xt, Xv, Yt, Yv = train_test_split(X, y, test_size =0.2, shuffle=False)

Reg = LogisticRegression()
Reg.fit(Xt, np.array(Yt))

In [ ]:
# Reg.predict_proba(Xv)[:,1]

In [ ]:
y_pred = Reg.predict_proba(Xv)[:,1]
#y_pred = np.array(Xv['answered_correctly_content'])
y_true = np.array(Yv)

In [ ]:
Reg.coef_

In [ ]:
y.value_counts()

In [ ]:
roc_auc_score(y_true, y_pred)

### Predict

In [ ]:
env = riiideducation.make_env()

iter_test = env.iter_test()

In [ ]:
iter_test

In [ ]:
for (test, sample_prediction_df) in iter_test:
    #uid = test['user_id']

    test = test.merge(usrCorrect.reset_index(), on='user_id', how='left')
    test = test.merge(problemCorrect.reset_index(), on='content_id', how='left')

    XTest = test[['answered_correctly_x','answered_correctly_y']]

    XTest.columns = ['usr','problem']

    XTest=XTest.fillna(0.5)

    output = Reg.predict_proba(XTest)[:,1]

    test['answered_correctly'] = output
    
    env.predict(test.loc[test['content_type_id']==0, ['row_id', 'answered_correctly']])